<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #08



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>

<a id='prob'></a>
## Problem #08

<br>
Facilities <br>
$|I| = 1000$ 
<br>
<br>
Clients <br>
$|J| = 1000$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [10]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Facilities Fixed Costs
def getFacilities_STRCapacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(file_list[i+shift])
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost


# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/MS1'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))

start = time.time()


# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()


end = time.time()
print("TIME IS: ",end - start)

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1001000 rows, 1001000 columns and 3000000 nonzeros
Model fingerprint: 0x653b48d2
Variable types: 1000000 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3.839986e+08
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve removed 0 rows and 0 columns (presolve time = 10s) ...
Presolve removed 0 rows and 0 columns (presolve time = 15s) ...
Presolve removed 0 rows and 0 columns (presolve time = 21s) ...
Presolve removed 0 rows and 0 columns (presolve time = 27s) ...
Presolve time: 26.71s
Presolved: 1001000 rows, 1001000 columns, 3000000 nonzeros
Variable type

In [2]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_08_Simplex.lp')


Build a warehouse at location 116.

Build a warehouse at location 166.

Build a warehouse at location 739.

Client 1 receives 100.0 % of its demand  from Warehouse 116 .

Client 2 receives 100.0 % of its demand  from Warehouse 739 .

Client 3 receives 100.0 % of its demand  from Warehouse 166 .

Client 4 receives 100.0 % of its demand  from Warehouse 739 .

Client 5 receives 100.0 % of its demand  from Warehouse 116 .

Client 6 receives 100.0 % of its demand  from Warehouse 116 .

Client 7 receives 100.0 % of its demand  from Warehouse 116 .

Client 8 receives 100.0 % of its demand  from Warehouse 166 .

Client 9 receives 100.0 % of its demand  from Warehouse 116 .

Client 10 receives 100.0 % of its demand  from Warehouse 116 .

Client 11 receives 100.0 % of its demand  from Warehouse 166 .

Client 12 receives 100.0 % of its demand  from Warehouse 116 .

Client 13 receives 100.0 % of its demand  from Warehouse 739 .

Client 14 receives 100.0 % of its demand  from Warehouse 116 .

Clie

<br>
<br>

<a id='random'></a>
### Random Heuristic 


In [3]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

start = time.time()

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



end = time.time()
print("TIME IS: ",end - start)

#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[0, 2, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 27, 28, 29, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 46, 47, 48, 49, 50, 51, 54, 56, 57, 58, 60, 62, 63, 64, 65, 69, 70, 72, 73, 75, 76, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 96, 99, 102, 103, 105, 106, 108, 110, 112, 113, 117, 118, 119, 120, 122, 124, 125, 126, 130, 132, 133, 134, 135, 137, 139, 140, 141, 144, 145, 147, 149, 150, 151, 152, 154, 155, 156, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 174, 176, 177, 180, 181, 183, 185, 186, 187, 189, 191, 192, 193, 195, 196, 197, 200, 202, 203, 205, 207, 209, 210, 212, 214, 215, 216, 217, 218, 220, 222, 225, 226, 228, 230, 231, 232, 235, 236, 239, 240, 241, 242, 247, 250, 257, 258, 262, 264, 266, 267, 268, 269, 270, 271, 272, 274, 280, 281, 282, 283, 284, 289, 291, 292, 294, 295, 296, 297, 298, 299, 300, 301, 304, 305, 307, 308, 309, 310, 312, 313, 314, 317, 318, 319, 321, 322, 324, 325, 326, 327, 332, 333, 334, 335, 336, 337, 338, 342, 345,

In [4]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 1.

Build a warehouse at location 3.

Build a warehouse at location 4.

Build a warehouse at location 5.

Build a warehouse at location 6.

Build a warehouse at location 9.

Build a warehouse at location 10.

Build a warehouse at location 11.

Build a warehouse at location 13.

Build a warehouse at location 14.

Build a warehouse at location 15.

Build a warehouse at location 16.

Build a warehouse at location 17.

Build a warehouse at location 18.

Build a warehouse at location 19.

Build a warehouse at location 20.

Build a warehouse at location 22.

Build a warehouse at location 24.

Build a warehouse at location 25.

Build a warehouse at location 26.

Build a warehouse at location 28.

Build a warehouse at location 29.

Build a warehouse at location 30.

Build a warehouse at location 32.

Build a warehouse at location 33.

Build a warehouse at location 34.

Build a warehouse at location 35.

Build a warehouse at location 36.

Build a warehouse at loca

In [5]:
len(facs)

628

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [6]:
#Get the lowest shipping cost for each Client

start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 18, 19, 20, 21, 25, 28, 29, 30, 32, 33, 34, 36, 38, 39, 41, 43, 44, 45, 47, 50, 51, 53, 54, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 69, 70, 72, 75, 77, 79, 80, 81, 82, 83, 87, 88, 89, 90, 93, 95, 96, 98, 99, 101, 102, 103, 104, 105, 106, 108, 111, 115, 116, 117, 120, 121, 122, 123, 124, 127, 129, 130, 134, 135, 136, 137, 138, 140, 141, 147, 150, 151, 152, 153, 155, 156, 157, 162, 165, 167, 168, 169, 170, 171, 172, 175, 176, 177, 178, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 194, 196, 198, 199, 200, 201, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 218, 219, 220, 221, 223, 224, 225, 226, 228, 230, 231, 232, 236, 237, 238, 239, 243, 244, 245, 248, 249, 251, 253, 255, 256, 258, 259, 260, 264, 265, 267, 268, 269, 270, 271, 273, 275, 276, 279, 280, 282, 284, 285, 286, 287, 289, 290, 291, 292, 294, 295, 296, 297, 301, 304, 305, 306, 307, 312, 316, 317, 318, 320, 322, 326, 328, 331, 332, 335, 337, 338, 339, 340, 341, 343, 3

<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [7]:
#Get the greatest shipping cost for each Client

start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[1, 2, 4, 6, 7, 10, 11, 13, 16, 17, 18, 19, 21, 22, 24, 26, 27, 29, 31, 32, 34, 35, 38, 40, 41, 42, 44, 46, 47, 48, 51, 53, 54, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 70, 72, 73, 76, 78, 79, 81, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 97, 99, 100, 102, 103, 104, 107, 109, 110, 111, 112, 115, 117, 119, 122, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 139, 141, 142, 144, 145, 147, 148, 149, 153, 154, 158, 159, 162, 165, 166, 167, 171, 172, 173, 175, 176, 177, 178, 179, 180, 182, 185, 188, 189, 190, 193, 195, 196, 197, 198, 199, 200, 202, 203, 206, 207, 208, 209, 210, 212, 213, 214, 216, 218, 219, 220, 221, 222, 223, 224, 226, 235, 236, 237, 238, 239, 241, 242, 243, 245, 248, 249, 250, 258, 259, 260, 261, 262, 263, 265, 266, 267, 269, 272, 274, 275, 276, 277, 278, 281, 282, 283, 284, 285, 286, 291, 293, 294, 295, 296, 299, 302, 303, 304, 305, 306, 307, 308, 309, 311, 312, 314, 316, 317, 319, 320, 324, 325, 326, 329, 330, 331, 332, 333, 334, 336, 338, 341, 343, 344, 345, 346, 34

<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [8]:
#Get the lowest shipping cost for each Client

start = time.time()


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

[738]
200
209092.5674779998
1
TIME IS:  2.2509641647338867


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [9]:
#Get the lowest shipping cost for each Client

start = time.time()


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

[611]
992988
993989173.7341818
1
TIME IS:  1.5452799797058105
